In [ ]:
# ============================================================
# TODO-EN-UNO: Lectura Excel + Cálculos + Dashboard + Análisis + PDF
# ============================================================
!pip -q install gradio plotly openpyxl statsmodels reportlab

import os, re, io, shutil, requests, numpy as np, pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
import plotly.graph_objects as go
import statsmodels.api as sm
from reportlab.lib.pagesizes import letter, landscape
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak, Image
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
import warnings
warnings.filterwarnings(
    "ignore",
    message="Data Validation extension is not supported and will be removed",
    category=UserWarning,
    module="openpyxl.worksheet._reader"
)
pd.options.display.float_format = '{:,.2f}'.format

# ==========================
# 0) CONFIG: ruta del Excel
# ==========================
# Si usas Drive, descomenta:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

EXCEL_PATH = '/content/drive/MyDrive/finanzas/PlantillaBC_2Grupo No. 1.xlsx'  # <-- cámbiala si hace falta

# ==========================
# 1) UTILIDADES DE PARSEO
# ==========================
def _to_number(x):
    if pd.isna(x): return np.nan
    if isinstance(x, (int, float, np.number)): return float(x)
    s = str(x).strip().replace(' ', '').replace('%','')
    if s.count(',') > 1 and '.' in s:
        s = s.replace(',', '')
    elif s.count('.') > 1 and ',' in s:
        s = s.replace('.', '').replace(',', '.')
    else:
        if ',' in s and '.' not in s:
            s = s.replace(',', '.')
        s = s.replace(',', '')
    try:
        return float(s)
    except:
        return np.nan

def parse_multi_year_sheet(path, sheet_name):
    """
    Detecta la fila 'CONCEPTOS' y columnas de montos por año (encabezado tipo fecha o encabezado 'FECHA' y la fecha debajo).
    Devuelve (df_wide con columnas ['Cuenta','YYYY'...], lista_años_ordenada)
    """
    df = pd.read_excel(path, sheet_name=sheet_name, header=None)
    hdr = None
    for r in range(min(50, df.shape[0])):
        if str(df.iat[r, 0]).strip().upper() == 'CONCEPTOS':
            hdr = r
            break
    if hdr is None:
        raise ValueError(f"No se encontró 'CONCEPTOS' en la hoja {sheet_name}")

    date_row = hdr + 1

    def _get_year_from(cell):
        dt = pd.to_datetime(cell, errors='coerce')
        if pd.isna(dt): return None
        return int(dt.year)

    amount_cols, year_labels = [], []
    for c in range(1, df.shape[1]):
        top = df.iat[hdr, c]
        below = df.iat[date_row, c] if date_row < df.shape[0] else None
        y = None
        if isinstance(top, str) and top.strip().upper() == 'FECHA':
            y = _get_year_from(below)
        else:
            y = _get_year_from(top)
        if y is not None:
            amount_cols.append(c)
            year_labels.append(str(y))

    if not amount_cols:
        raise ValueError(f"No se detectaron columnas de año en {sheet_name}")

    start = hdr + 2
    sub = df.iloc[start:, [0] + amount_cols].copy()
    temp_cols = ['Cuenta'] + [f"Y{i}_{y}" for i, y in enumerate(year_labels)]
    sub.columns = temp_cols
    sub['Cuenta'] = sub['Cuenta'].astype(str).str.strip()
    for c in temp_cols[1:]:
        sub[c] = sub[c].apply(_to_number)

    uniq_years = sorted(set(year_labels))
    out = pd.DataFrame({'Cuenta': sub['Cuenta']})
    for y in uniq_years:
        cols_y = [c for c in temp_cols[1:] if c.endswith(f"_{y}")]
        out[y] = sub[cols_y].sum(axis=1, skipna=True)
    out = out.dropna(how='all', subset=uniq_years).reset_index(drop=True)
    return out, uniq_years

def vertical_analysis_exact(df, year, total_patterns, prefer_max=True):
    out = df[['Cuenta', year]].copy()
    mask = False
    for pat in total_patterns:
        mask = mask | out['Cuenta'].str.contains(pat, case=False, regex=True, na=False)
    matches = out.loc[mask, year]
    if not matches.empty:
        total = (matches.max(skipna=True) if prefer_max else matches.sum(skipna=True))
    else:
        total = out[year].sum(skipna=True)
    out[f'%_{year}'] = np.where(total == 0, np.nan, out[year] / total * 100.0)
    return out, total

def horizontal_analysis(df, years_sorted):
    out = df[['Cuenta'] + years_sorted].copy()
    for i in range(1, len(years_sorted)):
        a, b = years_sorted[i-1], years_sorted[i]
        out[f'Var% {a}->{b}'] = np.where(out[a].fillna(0)==0, np.nan, (out[b]-out[a])/out[a]*100.0)
    return out

def pick_value(df, patterns, year, prefer_max=True):
    """Búsqueda robusta sin warnings de regex: literal si no hay metacaracteres; si hay, convierte '(' en '(?:'."""
    mask = pd.Series(False, index=df.index)
    for pat in patterns:
        looks_regex = bool(re.search(r'[.\^\$\*\+\?\{\}\[\]\|\(\)\\]', pat))
        if looks_regex:
            pat_nc = re.sub(r'\((?!\?)', '(?:', pat)
            m = df['Cuenta'].str.contains(pat_nc, case=False, regex=True, na=False)
        else:
            m = df['Cuenta'].str.contains(pat, case=False, regex=False, na=False)
        mask = mask | m
    vals = df.loc[mask, year]
    if vals.empty: return np.nan
    return vals.max(skipna=True) if prefer_max else vals.sum(skipna=True)

def sdiv(n, d):
    return np.nan if (d in [0, None] or pd.isna(d) or pd.isna(n) or d == 0) else n/d

def calculate_eva(year, nopat, capital_invertido, wacc):
    if any(pd.isna(v) for v in [nopat, capital_invertido, wacc]):
        return np.nan
    return nopat - capital_invertido * wacc
wacc_dec = np.nan

# ==========================
# 2) LECTURA Y CÁLCULOS
# ==========================
balance_wide, bal_years = parse_multi_year_sheet(EXCEL_PATH, 'ESTRUCTURA FINANCIERA')
eres_wide,    er_years  = parse_multi_year_sheet(EXCEL_PATH, 'ESTRUCTURA ECONOMICA')

# Verticales
balance_vertical_all = []
for y in bal_years:
    vb, _ = vertical_analysis_exact(balance_wide, y, [r'^TOTAL\s+ACTIVOS?$'], prefer_max=True)
    balance_vertical_all.append(vb)

er_vertical_all = []
for y in er_years:
    ve, _ = vertical_analysis_exact(
        eres_wide, y,
        [r'^\s*\.?\s*=\s*INGRESOS\s+TOTALES\s*$', r'^\s*VENTAS\s*$', r'^\s*INGRESOS\s+NETOS\s*$'],
        prefer_max=True
    )
    er_vertical_all.append(ve)

# Horizontales
balance_horizontal = horizontal_analysis(balance_wide, bal_years)
eres_horizontal    = horizontal_analysis(eres_wide,    er_years)

# Ratios (años comunes)
years_common = sorted(set(bal_years).intersection(set(er_years)))
ratios = pd.DataFrame(index=years_common)
for y in years_common:
    AT  = pick_value(balance_wide, [r'^TOTAL\s+ACTIVOS?$'], y)
    PT  = pick_value(balance_wide, [r'^TOTAL\s+PASIVOS?$'], y)
    PAT = pick_value(balance_wide, [r'RECURSOS\s+PROPIOS', r'^PATRIMONIO$', r'CAPITAL\s+CONTABLE', r'FONDOS\s+PROPIOS'], y)
    AC  = pick_value(balance_wide, [r'^ACTIVO\s+(CIRCULANTE|CORRIENTE)$'], y)
    PC  = pick_value(balance_wide, [r'^PASIVO\s+(CIRCULANTE|CORRIENTE)$'], y)
    INV = pick_value(balance_wide, [r'INVENTARIOS?$', r'^EXISTENCIAS$'], y, prefer_max=False)
    VN  = pick_value(eres_wide, [r'^\s*\.?\s*=\s*INGRESOS\s+TOTALES\s*$', r'^\s*VENTAS\s*$', r'^\s*INGRESOS\s+NETOS\s*$'], y)
    UN  = pick_value(eres_wide, [r'UTILIDAD\s+NETA$', r'BENEFICIO\s+NETO$', r'GANANCIA\s+NETA$'], y)
    EBIT = pick_value(eres_wide, [r'UTILIDAD\s+OPERATIVA', r'EBIT'], y)
    IMP  = pick_value(eres_wide, [r'IMPUESTOS', r'ISR'], y)
    NOPAT = (EBIT - IMP) if not (pd.isna(EBIT) or pd.isna(IMP)) else np.nan
    CI    = (AT - PC) if not (pd.isna(AT) or pd.isna(PC)) else np.nan


    ratios.loc[y, 'Liquidez Corriente']                  = sdiv(AC, PC)
    ratios.loc[y, 'Prueba Ácida']                        = sdiv((AC - (0 if pd.isna(INV) else INV)), PC)
    ratios.loc[y, 'Endeudamiento (Pasivo/Activo)']       = sdiv(PT, AT)
    ratios.loc[y, 'Apalancamiento (Activo/Patrimonio)']  = sdiv(AT, PAT)
    ratios.loc[y, 'Margen Neto']                         = sdiv(UN, VN)
    ratios.loc[y, 'ROA']                                 = sdiv(UN, AT)
    ratios.loc[y, 'ROE']                                 = sdiv(UN, PAT)
    ratios.loc[y, 'Rotación de Activos']                 = sdiv(VN, AT)
    ratios.loc[y, 'NOPAT']                               = NOPAT
    ratios.loc[y, 'Capital Invertido']                   = CI
    if not pd.isna(wacc_dec):
        ratios.loc[y, 'EVA']                     = calculate_eva(y, NOPAT, CI, wacc_dec)



# ==========================
# 3) ARCHIVOS BASE
# ==========================
xlsx_out = "/content/analisis_financiero_salida.xlsx"
with pd.ExcelWriter(xlsx_out, engine='openpyxl') as writer:
    balance_wide.to_excel(writer, sheet_name='Balance_Wide', index=False)
    eres_wide.to_excel(writer, sheet_name='ER_Wide', index=False)
    balance_horizontal.to_excel(writer, sheet_name='Balance_Horizontal', index=False)
    eres_horizontal.to_excel(writer, sheet_name='ER_Horizontal', index=False)
    ratios.reset_index(names='Año').to_excel(writer, sheet_name='Ratios', index=False)

csv_balance_h = "/content/balance_horizontal_descarga.csv"; balance_horizontal.to_csv(csv_balance_h, index=False)
csv_er_h      = "/content/er_horizontal_descarga.csv";     eres_horizontal.to_csv(csv_er_h, index=False)
csv_ratios    = "/content/ratios_descarga.csv";            ratios.to_csv(csv_ratios)


# ==========================
# 4) EXTERNOS (WDI/TPM) + PANEL
# ==========================
FACTOR_CANDIDATES = ["Inflación_%","PIB_real_%","USD/DOP","TPM_%"]

def fetch_wb_series(country_code, indicator_code):
    url = f"https://api.worldbank.org/v2/country/{country_code}/indicator/{indicator_code}?format=json&per_page=20000"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    data = r.json()
    if not isinstance(data, list) or len(data) < 2 or data[1] is None:
        return pd.Series(dtype=float)
    rows, out = data[1], {}
    for row in rows:
        yr, val = row.get("date"), row.get("value")
        if yr and val is not None:
            out[int(yr)] = float(val)
    return pd.Series(out).sort_index()

def build_external(years_all, country_code="DOM", tpm_dict=None):
    y0, y1 = min(years_all), max(years_all)
    ext = pd.DataFrame({"Año": range(y0-1, y1+1)})
    cpi_yoy   = fetch_wb_series(country_code, "FP.CPI.TOTL.ZG")
    gdp_yoy   = fetch_wb_series(country_code, "NY.GDP.MKTP.KD.ZG")
    fx_lcuusd = fetch_wb_series(country_code, "PA.NUS.FCRF")
    ext["Inflación_%"] = ext["Año"].map(cpi_yoy.to_dict())
    ext["PIB_real_%"]  = ext["Año"].map(gdp_yoy.to_dict())
    ext["USD/DOP"]     = ext["Año"].map(fx_lcuusd.to_dict())
    if tpm_dict is None: tpm_dict = {}
    ext["TPM_%"]       = ext["Año"].map(tpm_dict)
    return ext.set_index("Año").sort_index()

def _get_year_col(df: pd.DataFrame, year):
    ys = str(year)
    for c in df.columns:
        if str(c).strip() == ys: return c
    return None

def pick_value_flex(df: pd.DataFrame, patterns, year, prefer_max=True):
    if df is None or 'Cuenta' not in df.columns: return np.nan
    col = _get_year_col(df, year)
    if col is None: return np.nan
    mask = False
    for pat in patterns:
        mask = mask | df['Cuenta'].astype(str).str.contains(pat, case=False, regex=True, na=False)
    vals = df.loc[mask, col]
    if vals.empty: return np.nan
    return vals.max(skipna=True) if prefer_max else vals.sum(skipna=True)

def build_panel(years_all, external):
    ventas = pd.Series({y: pick_value_flex(eres_wide,
                                           [r'^\s*\.?\s*=\s*INGRESOS\s+TOTALES\s*$', r'^\s*VENTAS\s*$', r'^\s*INGRESOS\s+NETOS\s*$'],
                                           y) for y in years_all}, name="Ventas")
    util_neta = pd.Series({y: pick_value_flex(eres_wide,
                                              [r'UTILIDAD\s+NETA$', r'BENEFICIO\s+NETO$', r'GANANCIA\s+NETA$'],
                                              y) for y in years_all}, name="Utilidad Neta")
    panel = pd.DataFrame({"Ventas": ventas, "Utilidad Neta": util_neta})
    rcopy = ratios.copy()
    try: rcopy.index = rcopy.index.astype(int)
    except: pass
    panel = panel.join(rcopy, how="left")
    ext_idx = external if external.index.name=="Año" else external.set_index("Año")
    panel = panel.join(ext_idx, how="left").sort_index()
    panel["Ventas_YoY_%"]   = panel["Ventas"].astype(float).pct_change(periods=1, fill_method=None) * 100
    panel["UtilNeta_YoY_%"] = panel["Utilidad Neta"].astype(float).pct_change(periods=1, fill_method=None) * 100
    for col in ["Inflación_%","PIB_real_%","USD/DOP","TPM_%"]:
        if col in panel.columns:
            panel[f"{col}_lag1"] = panel[col].shift(1)
    return panel

years_all = sorted({*map(int, bal_years), *map(int, er_years)})
external_df = build_external(years_all, country_code="DOM", tpm_dict={})
panel_df    = build_panel(years_all, external_df)

# CSV para descargas
external_csv = "/content/factores_externos_descarga.csv"; external_df.to_csv(external_csv)
panel_csv    = "/content/panel_interno_externo_descarga.csv"; panel_df.to_csv(panel_csv)

# ==========================
# 5) ANALÍTICA (comparadas, heatmap, OLS)
# ==========================
def zscore(s: pd.Series):
    s = s.astype(float); std = s.std(ddof=0)
    return (s - s.mean())/std if std and not np.isnan(std) and std != 0 else s*0

def ts_compare(factor_col, kpi_col, lag=0, normalize=False):
    df = panel_df.copy()
    if factor_col not in df.columns or kpi_col not in df.columns:
        return go.Figure(), go.Figure(), pd.DataFrame(), "Columnas inválidas."
    fx = df[factor_col].shift(lag) if lag else df[factor_col]
    comp = pd.DataFrame({kpi_col: df[kpi_col], factor_col: fx}).dropna()
    if comp.empty:
        return go.Figure(), go.Figure(), pd.DataFrame(), "Sin datos tras filtros/lag."
    kpi_plot = zscore(comp[kpi_col]) if normalize else comp[kpi_col]
    fac_plot = zscore(comp[factor_col]) if normalize else comp[factor_col]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=comp.index.astype(str), y=kpi_plot, mode="lines+markers", name=kpi_col))
    fig.add_trace(go.Scatter(x=comp.index.astype(str), y=fac_plot, mode="lines+markers",
                             name=f"{factor_col}" + (f" (lag {lag})" if lag else ""), yaxis="y2"))
    fig.update_layout(title=f"{kpi_col} vs {factor_col}" + (f" (lag {lag})" if lag else ""),
                      xaxis_title="Año",
                      yaxis_title=kpi_col + (" (z-score)" if normalize else ""),
                      yaxis2=dict(title=factor_col + (" (z-score)" if normalize else ""), overlaying='y', side='right'),
                      template="plotly_white")
    scatter = go.Figure()
    x, y = comp[factor_col].values, comp[kpi_col].values
    scatter.add_trace(go.Scatter(x=x, y=y, mode="markers+text", text=comp.index.astype(str),
                                 textposition="top center", name="Obs"))
    try:
        coef = np.polyfit(x, y, 1); x_fit = np.linspace(x.min(), x.max(), 50)
        y_fit = coef[0]*x_fit + coef[1]; y_hat = coef[0]*x + coef[1]
        r2 = 1 - np.sum((y - y_hat)**2)/np.sum((y - y.mean())**2)
        scatter.add_trace(go.Scatter(x=x_fit, y=y_fit, mode="lines", name=f"Ajuste (R²={r2:.3f})"))
    except Exception:
        pass
    scatter.update_layout(title=f"Dispersión: {kpi_col} vs {factor_col}",
                          xaxis_title=factor_col, yaxis_title=kpi_col, template="plotly_white")
    corr = comp[kpi_col].corr(comp[factor_col])
    info = f"**Correlación (Pearson)** {kpi_col} vs {factor_col}" + (f" (lag {lag})" if lag else "") + f": **{corr:.3f}**"
    return fig, scatter, comp.reset_index(names="Año"), info

def corr_heatmap(lag=0):
    KPI_COLS = [c for c in ["Ventas_YoY_%","UtilNeta_YoY_%","Margen Neto","ROE","ROA","Rotación de Activos"] if c in panel_df.columns]
    FACTORS  = [c for c in FACTOR_CANDIDATES if c in panel_df.columns]
    if not KPI_COLS or not FACTORS: return go.Figure(), pd.DataFrame()
    mat = pd.DataFrame(index=KPI_COLS, columns=FACTORS, dtype=float)
    for k in KPI_COLS:
        for f in FACTORS:
            s = pd.concat([panel_df[k], panel_df[f].shift(lag) if lag else panel_df[f]], axis=1).dropna()
            mat.loc[k, f] = s.iloc[:,0].corr(s.iloc[:,1])
    fig = go.Figure(data=go.Heatmap(z=mat.values, x=mat.columns.tolist(), y=mat.index.tolist(),
                                    zmin=-1, zmax=1, colorscale="RdBu"))
    fig.update_layout(title=f"Heatmap de correlaciones (lag factores = {lag})", template="plotly_white")
    return fig, mat.reset_index()

def run_ols(dep, indeps):
    if dep not in panel_df.columns or not indeps:
        return "Selecciona dependiente y factores."
    df = panel_df[[dep] + indeps].dropna().copy()
    if df.empty or df.shape[0] < len(indeps)+2:
        return "No hay suficientes observaciones."
    y = df[dep].astype(float); X = sm.add_constant(df[indeps].astype(float), has_constant='add')
    model = sm.OLS(y, X).fit()
    out = pd.DataFrame({
        "Variable": ["Constante"] + indeps,
        "Coef": model.params.values.round(4),
        "StdErr": model.bse.values.round(4),
        "t": model.tvalues.values.round(3),
        "p>|t|": model.pvalues.values.round(4)
    })
    meta = f"R²={model.rsquared:.3f} | R² ajustado={model.rsquared_adj:.3f} | N={int(model.nobs)}"
    out_csv = "/content/ols_results.csv"; out.to_csv(out_csv, index=False)
    return out, meta, out_csv

# ==========================
# 6) ÍNDICE PESTEL
# ==========================
def _scale_to_score(x, lo, hi, pos_good=True):
    if x is None or (isinstance(x, float) and np.isnan(x)): return np.nan
    x_clip = min(max(x, lo), hi); u = (x_clip - lo) / (hi - lo + 1e-9)
    if not pos_good: u = 1 - u
    return (u * 2 - 1) * 100

def _classify(score):
    if score <= -15: return "Muy crítico"
    if score <=  -5: return "Crítico"
    if score <=   5: return "Neutral"
    if score <=  15: return "Bueno"
    return "Excelente"

def _econ_auto_score(year):
    ext = external_df.copy()
    gdp = ext.at[year, "PIB_real_%"] if "PIB_real_%" in ext.columns and year in ext.index else np.nan
    inf = ext.at[year, "Inflación_%"] if "Inflación_%" in ext.columns and year in ext.index else np.nan
    tpm = ext.at[year, "TPM_%"]       if "TPM_%" in ext.columns and year in ext.index else np.nan
    fx_score = np.nan
    if "USD/DOP" in ext.columns and year in ext.index and (year-1) in ext.index:
        fx, fx_prev = ext.at[year, "USD/DOP"], ext.at[year-1, "USD/DOP"]
        dep_yoy = (fx/fx_prev - 1) * 100 if fx_prev not in (0, np.nan, None) else np.nan
        fx_score = _scale_to_score(dep_yoy, lo=-5, hi=30, pos_good=False)
    s_gdp = _scale_to_score(gdp, lo=-5, hi=10, pos_good=True)
    s_inf = _scale_to_score(inf, lo=0,  hi=20, pos_good=False)
    s_tpm = _scale_to_score(tpm, lo=0,  hi=20, pos_good=False)
    vals = [v for v in [s_gdp, s_inf, fx_score, s_tpm] if not np.isnan(v)]
    return float(np.mean(vals)) if vals else np.nan

def env_index_calc(year, econ_w, soc_w, geo_w, pol_w, tec_w, cul_w,
                   auto_econ, econ_manual, soc_res, geo_res, pol_res, tec_res, cul_res):
    pesos = {"Económico":econ_w, "Social":soc_w, "Geográfico":geo_w, "Político":pol_w, "Tecnológico":tec_w, "Cultural":cul_w}
    w_sum = sum(pesos.values()) or 1.0
    pesos_norm = {k: float(v)/w_sum for k,v in pesos.items()}
    econ_res = _econ_auto_score(int(year)) if auto_econ else float(econ_manual)
    res = {"Económico":econ_res, "Social":float(soc_res), "Geográfico":float(geo_res),
           "Político":float(pol_res), "Tecnológico":float(tec_res), "Cultural":float(cul_res)}
    rows = []
    for k in ["Económico","Social","Geográfico","Político","Tecnológico","Cultural"]:
        aporte = pesos_norm[k] * (res[k] if not np.isnan(res[k]) else 0.0)
        rows.append([k, round(pesos[k],2), round(pesos_norm[k]*100,2), None if np.isnan(res[k]) else round(res[k],2), round(aporte,2)])
    df = pd.DataFrame(rows, columns=["Categoría","Peso % (input)","Peso % (normalizado)","Resultado %","Aporte ponderado %"])
    score = float(np.nansum([r[-1] for r in rows])); clas  = _classify(score)
    fig = go.Figure()
    fig.add_trace(go.Bar(x=df["Categoría"], y=df["Resultado %"], name="Resultado (%)"))
    fig.add_trace(go.Bar(x=df["Categoría"], y=df["Aporte ponderado %"], name="Aporte ponderado (pp)"))
    fig.update_layout(barmode="group", title=f"Índice de Entorno ({year}) — Total {score:.1f}% · {clas}",
                      xaxis_title="Categoría", yaxis_title="Porcentaje / Puntos ponderados", template="plotly_white")
    out_csv = "/content/indice_entorno.csv"; df.assign(Total_Indice=score, Clasificacion=clas).to_csv(out_csv, index=False)
    return fig, df, f"**Índice total {year}: {score:.1f}% → {clas}**", out_csv, (None if np.isnan(econ_res) else float(econ_res))


# ==========================
# 7) PDF (sin OpenAI)
# ==========================
def is_pct_col(colname: str) -> bool:
    return (str(colname).startswith('%_') or str(colname).startswith('Var%')
            or str(colname) in ['Margen Neto','ROA','ROE','Endeudamiento (Pasivo/Activo)'])

def fmt_val(colname, x):
    if pd.isna(x): return ""
    if isinstance(x, (int, float, np.number)):
        return f"{x*100:,.2f}%" if (is_pct_col(colname) and abs(x) <= 2) else (f"{x:,.2f}%" if is_pct_col(colname) else f"{x:,.2f}")
    return str(x)

def df_to_tabledata(df, max_rows=30):
    d = df.copy()
    if d.index.name or not isinstance(d.index, pd.RangeIndex):
        d = d.reset_index()
    if len(d) > max_rows:
        d = d.head(max_rows)
    header = [str(c) for c in d.columns.tolist()]
    rows = []
    for _, row in d.iterrows():
        rows.append([fmt_val(col, row[col]) for col in d.columns])
    return [header] + rows

def styled_table(data, colWidths=None):
    t = Table(data, colWidths=colWidths)
    t.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor('#4a5568')),
        ('TEXTCOLOR',  (0,0), (-1,0), colors.whitesmoke),
        ('ALIGN',      (0,0), (-1,0), 'CENTER'),
        ('FONTNAME',   (0,0), (-1,0), 'Helvetica-Bold'),
        ('FONTSIZE',   (0,0), (-1,0), 9),
        ('BOTTOMPADDING', (0,0), (-1,0), 6),
        ('ALIGN', (0,1), (-1,-1), 'RIGHT'),
        ('ALIGN', (0,1), (0,-1), 'LEFT'),
        ('FONTSIZE', (0,1), (-1,-1), 8),
        ('ROWBACKGROUNDS', (0,1), (-1,-1), [colors.whitesmoke, colors.HexColor('#f7fafc')]),
        ('GRID', (0,0), (-1,-1), 0.25, colors.grey),
    ]))
    return t

def save_line_png(series, title, fname, ylabel=''):
    s = pd.Series(series).dropna()
    if s.empty: return None
    plt.figure()
    plt.plot(list(map(str, s.index)), s.values, marker='o')
    plt.title(title); plt.xlabel('Año'); plt.ylabel(ylabel); plt.grid(True)
    out = f"/content/{fname}.png"
    plt.savefig(out, bbox_inches='tight', dpi=140); plt.close()
    return out

def generate_pdf():
    yrs = sorted(set(bal_years).intersection(set(er_years)))
    if not yrs: return None
    # series clave
    VN = pd.Series({y: pick_value_flex(eres_wide,
        [r'^\s*\.?\s*=\s*INGRESOS\s+TOTALES\s*$', r'^\s*VENTAS\s*$', r'^\s*INGRESOS\s+NETOS\s*$'], y) for y in yrs})
    UN = pd.Series({y: pick_value_flex(eres_wide,
        [r'UTILIDAD\s+NETA$', r'BENEFICIO\s+NETO$', r'GANANCIA\s+NETA$'], y) for y in yrs})
    AT = pd.Series({y: pick_value_flex(balance_wide, [r'^TOTAL\s+ACTIVOS?$'], y) for y in yrs})
    PT = pd.Series({y: pick_value_flex(balance_wide, [r'^TOTAL\s+PASIVOS?$'], y) for y in yrs})
    PAT= pd.Series({y: pick_value_flex(balance_wide,
        [r'RECURSOS\s+PROPIOS', r'^PATRIMONIO$', r'CAPITAL\s+CONTABLE', r'FONDOS\s+PROPIOS'], y) for y in yrs})

    imgs = []
    for s, title, fn, yl in [
        (VN,"Ingresos/Ventas","plot_ventas","Monto"),
        (UN,"Utilidad Neta","plot_utilidad","Monto"),
        (AT,"Total Activos","plot_activos","Monto"),
        (PT,"Total Pasivos","plot_pasivos","Monto"),
        (PAT,"Patrimonio","plot_patrimonio","Monto"),
    ]:
        p = save_line_png(s, title, fn, yl);  imgs.append(p) if p else None
    if 'Liquidez Corriente' in ratios.columns:
        p = save_line_png(ratios['Liquidez Corriente'], "Liquidez Corriente", "plot_liquidez", "Veces");  imgs.append(p) if p else None
    if 'Endeudamiento (Pasivo/Activo)' in ratios.columns:
        p = save_line_png(ratios['Endeudamiento (Pasivo/Activo)'], "Endeudamiento", "plot_endeuda", "Proporción"); imgs.append(p) if p else None
    if 'Margen Neto' in ratios.columns:
        p = save_line_png(ratios['Margen Neto'], "Margen Neto", "Proporción"); imgs.append(p) if p else None

    pdf_path = "/content/analisis_financiero.pdf"
    doc = SimpleDocTemplate(pdf_path, pagesize=landscape(letter),
                            leftMargin=24, rightMargin=24, topMargin=24, bottomMargin=24)
    styles = getSampleStyleSheet(); H1, H2, BODY = styles["Title"], styles["Heading2"], styles["BodyText"]
    story = []
    story.append(Paragraph("📊 Análisis Financiero — Resumen", H1))
    story.append(Spacer(1, 8))
    story.append(Paragraph(f"Años Balance: {', '.join(bal_years)}", BODY))
    story.append(Paragraph(f"Años E. Resultados: {', '.join(er_years)}", BODY))

    # pequeño resumen determinístico
    try:
        g_ing = (VN.iloc[-1]/VN.iloc[0]-1)*100 if VN.iloc[0] else np.nan
        g_un  = (UN.iloc[-1]/UN.iloc[0]-1)*100 if UN.iloc[0] else np.nan
        liq   = ratios.iloc[-1]['Liquidez Corriente'] if 'Liquidez Corriente' in ratios.columns else np.nan
        ende  = ratios.iloc[-1]['Endeudamiento (Pasivo/Activo)'] if 'Endeudamiento (Pasivo/Activo)' in ratios.columns else np.nan
        story.append(Spacer(1, 6))
        story.append(Paragraph(
            f"<b>Resumen:</b> Ingresos {g_ing:,.2f}% desde {yrs[0]} a {yrs[-1]}; "
            f"Utilidad Neta {g_un:,.2f}%. Liquidez Corriente {liq:,.2f}×; "
            f"Endeudamiento {ende*100:,.2f}%.", BODY
        ))
    except Exception:
        pass

    if imgs:
        story.append(PageBreak()); story.append(Paragraph("Gráficos de tendencia", H2))
        for p in imgs:
            story.append(Image(p, width=520, height=280)); story.append(Spacer(1, 6))

    # Tablas
    def clean_nan_rows(df):
        out = df.copy()
        out = out[out['Cuenta'].notna()]
        mask_notnan = out['Cuenta'].astype(str).str.strip().str.lower().ne('nan')
        return out[mask_notnan].reset_index(drop=True)

    for i, y in enumerate(bal_years):
        story.append(PageBreak())
        story.append(Paragraph(f"Análisis Vertical — Balance General ({y})", H2))
        story.append(styled_table(df_to_tabledata(balance_vertical_all[i], max_rows=28), colWidths=[280, 140, 120]))
    for i, y in enumerate(er_years):
        story.append(PageBreak())
        story.append(Paragraph(f"Análisis Vertical — Estado de Resultados ({y})", H2))
        story.append(styled_table(df_to_tabledata(er_vertical_all[i], max_rows=28), colWidths=[280, 140, 120]))
    story.append(PageBreak()); story.append(Paragraph("Análisis Horizontal — Balance General", H2))
    story.append(styled_table(df_to_tabledata(clean_nan_rows(balance_horizontal), max_rows=32)))
    story.append(PageBreak()); story.append(Paragraph("Análisis Horizontal — Estado de Resultados", H2))
    story.append(styled_table(df_to_tabledata(clean_nan_rows(eres_horizontal), max_rows=32)))
    story.append(PageBreak()); story.append(Paragraph("Ratios Financieros por Año", H2))
    rat_out = ratios.copy(); rat_out.index.name = 'Año'
    story.append(styled_table(df_to_tabledata(rat_out, max_rows=9999)))
    doc.build(story)
    return pdf_path

# ==========================
# 8) DASHBOARD ÚNICO
# ==========================
balance_vertical_by_year = {y: balance_vertical_all[i] for i, y in enumerate(bal_years)}
er_vertical_by_year      = {y: er_vertical_all[i]      for i, y in enumerate(er_years)}
ratio_options = list(ratios.columns) if not ratios.empty else []
year_opts_bal = list(balance_vertical_by_year.keys())
year_opts_er  = list(er_vertical_by_year.keys())
factors_available = [c for c in FACTOR_CANDIDATES if c in panel_df.columns]
kpi_available     = [c for c in ["Ventas_YoY_%","UtilNeta_YoY_%","Margen Neto","ROE","ROA","Rotación de Activos"] if c in panel_df.columns]

with gr.Blocks(title="Dashboard Financiero — Unificado") as demo:
    gr.Markdown("## 📊 Dashboard Financiero — Interno + Externo (WDI/TPM) + PESTEL + PDF (sin OpenAI)")

    # Ratios
    with gr.Tab("Ratios"):
        ratio_dd   = gr.Dropdown(ratio_options, value=(ratio_options[0] if ratio_options else None), label="Selecciona un ratio")
        ratio_plot = gr.Plot(label="Tendencia del ratio")
        ratio_tbl  = gr.Dataframe(value=ratios.reset_index().rename(columns={"index":"Año"}) if not ratios.empty else pd.DataFrame(),
                                  interactive=False, label="Tabla de ratios")
        with gr.Row():
            wacc_in = gr.Number(value=None, label="WACC (%)")
            kd_in   = gr.Number(value=None, label="Costo deuda (%)")
            ke_in   = gr.Number(value=None, label="Costo capital (%)")
            wacc_btn = gr.Button("Aplicar WACC")
        def _view_ratio(rname):
            if rname in ratios.columns:
                s = ratios[rname]; fig = go.Figure()
                fig.add_trace(go.Scatter(x=list(map(str, s.index)), y=s.values, mode="lines+markers", name=rname))
                fig.update_layout(title=rname, xaxis_title="Año", yaxis_title="Valor", template="plotly_white")
                return fig
            return go.Figure()
        gr.Button("Actualizar").click(_view_ratio, inputs=[ratio_dd], outputs=[ratio_plot])
        def _apply_wacc(wacc, kd, ke):
            global ratios
            wacc_val = wacc
            if (wacc_val is None or pd.isna(wacc_val)) and not (pd.isna(kd) or pd.isna(ke)):
                try:
                    last_y = ratios.index[-1]
                    d = pick_value(balance_wide, [r'^TOTAL\s+PASIVOS?$'], last_y)
                    e = pick_value(balance_wide, [r'RECURSOS\s+PROPIOS', r'^PATRIMONIO$', r'CAPITAL\s+CONTABLE', r'FONDOS\s+PROPIOS'], last_y)
                    total = d + e
                    wacc_val = sdiv(e, total) * ke + sdiv(d, total) * kd
                except Exception:
                    wacc_val = np.nan
            if wacc_val is None or pd.isna(wacc_val):
                return ratios.reset_index().rename(columns={"index":"Año"})
            global wacc_dec
            wacc_dec = wacc_val / 100.0
            for y in ratios.index:
                ratios.loc[y, 'EVA'] = calculate_eva(int(y), ratios.loc[y, 'NOPAT'], ratios.loc[y, 'Capital Invertido'], wacc_dec)
            ratios.to_csv(csv_ratios)
            with pd.ExcelWriter(xlsx_out, engine='openpyxl') as writer:
                balance_wide.to_excel(writer, sheet_name='Balance_Wide', index=False)
                eres_wide.to_excel(writer, sheet_name='ER_Wide', index=False)
                balance_horizontal.to_excel(writer, sheet_name='Balance_Horizontal', index=False)
                eres_horizontal.to_excel(writer, sheet_name='ER_Horizontal', index=False)
                ratios.reset_index(names='Año').to_excel(writer, sheet_name='Ratios', index=False)
            return ratios.reset_index().rename(columns={"index":"Año"})
        wacc_btn.click(_apply_wacc, inputs=[wacc_in, kd_in, ke_in], outputs=[ratio_tbl])

    # Balance Horizontal
    with gr.Tab("Balance Horizontal"):
        gr.Markdown("### Balance General — Horizontal (YoY)")
        gr.Dataframe(value=balance_horizontal, interactive=False)
        gr.DownloadButton("Descargar CSV", value=csv_balance_h)

    # ER Horizontal
    with gr.Tab("E.R. Horizontal"):
        gr.Markdown("### Estado de Resultados — Horizontal (YoY)")
        gr.Dataframe(value=eres_horizontal, interactive=False)
        gr.DownloadButton("Descargar CSV", value=csv_er_h)

    # Verticales
    with gr.Tab("Vertical Balance"):
        year_bal  = gr.Dropdown(year_opts_bal, value=(year_opts_bal[0] if year_opts_bal else None), label="Año")
        vbal_plot = gr.Plot(); vbal_tbl = gr.Dataframe(interactive=False)
        def _view_vbal(y):
            df = balance_vertical_by_year.get(y, pd.DataFrame());
            if df is None or df.empty: return go.Figure(), pd.DataFrame()
            col_pct = f"%_{y}"
            tmp = df[['Cuenta', col_pct]].dropna()
            tmp = tmp[~tmp['Cuenta'].astype(str).str.contains('TOTAL', case=False, na=False)]
            tmp = tmp.sort_values(col_pct, ascending=False).head(10)
            fig = go.Figure(go.Bar(x=tmp[col_pct].values, y=tmp['Cuenta'].values, orientation='h'))
            fig.update_layout(title=f"Top 10 partidas por porcentaje ({y})",
                              xaxis_title="Porcentaje (%)", yaxis_title="", template="plotly_white")
            return fig, df
        gr.Button("Ver").click(_view_vbal, inputs=[year_bal], outputs=[vbal_plot, vbal_tbl])

    with gr.Tab("Vertical E.R."):
        year_er  = gr.Dropdown(year_opts_er, value=(year_opts_er[0] if year_opts_er else None), label="Año")
        ver_plot = gr.Plot(); ver_tbl = gr.Dataframe(interactive=False)
        def _view_ver(y):
            df = er_vertical_by_year.get(y, pd.DataFrame());
            if df is None or df.empty: return go.Figure(), pd.DataFrame()
            col_pct = f"%_{y}"
            tmp = df[['Cuenta', col_pct]].dropna()
            tmp = tmp[~tmp['Cuenta'].astype(str).str.contains('TOTAL', case=False, na=False)]
            tmp = tmp.sort_values(col_pct, ascending=False).head(10)
            fig = go.Figure(go.Bar(x=tmp[col_pct].values, y=tmp['Cuenta'].values, orientation='h'))
            fig.update_layout(title=f"Top 10 conceptos por porcentaje ({y})",
                              xaxis_title="Porcentaje (%)", yaxis_title="", template="plotly_white")
            return fig, df
        gr.Button("Ver").click(_view_ver, inputs=[year_er], outputs=[ver_plot, ver_tbl])

    # Descargas base
    with gr.Tab("Descargas"):
        gr.Markdown("- **Excel** con hojas: Balance_Wide, ER_Wide, horizontales y ratios.")
        gr.DownloadButton("Descargar Excel (XLSX)", value=xlsx_out)
        gr.DownloadButton("Ratios (CSV)", value=csv_ratios)

    # 1) Factores externos
    with gr.Tab("1) Factores externos"):
        gr.Markdown("**WDI (World Bank)** — país ISO3 (DOM, USA, MEX...).")
        with gr.Row():
            cc_in = gr.Textbox(value="DOM", label="Código de país (ISO3)", scale=1)
            btn_wdi = gr.Button("Actualizar desde WDI", scale=0)
        wdi_info = gr.Markdown()
        ext_tbl  = gr.Dataframe(value=external_df.reset_index(), interactive=False, label="Factores externos")
        pan_tbl  = gr.Dataframe(value=panel_df.reset_index().rename(columns={"index":"Año"}), interactive=False, label="Panel interno–externo")

        def refresh_wdi(cc):
            global external_df, panel_df
            try:
                tpm_dict = external_df["TPM_%"].dropna().to_dict() if "TPM_%" in external_df.columns else {}
                external_df = build_external(years_all, country_code=cc, tpm_dict=tpm_dict)
                panel_df    = build_panel(years_all, external_df)
                external_df.to_csv(external_csv); panel_df.to_csv(panel_csv)
                return (f"Actualizado WDI para {cc}. Filas externos: {len(external_df)}. Años panel: {len(panel_df)}.",
                        external_df.reset_index(), panel_df.reset_index())
            except Exception as e:
                return (f"Error actualizando WDI: {e}", external_df.reset_index(), panel_df.reset_index())

        btn_wdi.click(refresh_wdi, inputs=[cc_in], outputs=[wdi_info, ext_tbl, pan_tbl])

        gr.Markdown("---")
        gr.Markdown("**TPM (BCRD) manual** — edita y guarda (columna `TPM_%`).")
        tpm_editor = gr.Dataframe(
            value=pd.DataFrame({"Año": external_df.index, "TPM_%": external_df.get("TPM_%")}).reset_index(drop=True),
            headers=["Año","TPM_%"], interactive=True, label="Editor TPM (anual)"
        )
        btn_save_tpm = gr.Button("Guardar TPM")

        def save_tpm(tpm_df_):
            global external_df, panel_df
            try:
                tmp = pd.DataFrame(tpm_df_, columns=["Año","TPM_%"]).dropna(subset=["Año"])
                tmp["Año"] = tmp["Año"].astype(int)
                ext_new = external_df.copy()
                for _, r in tmp.iterrows():
                    ext_new.loc[int(r["Año"]), "TPM_%"] = float(r["TPM_%"]) if pd.notna(r["TPM_%"]) else np.nan
                external_df = ext_new.sort_index()
                panel_df    = build_panel(years_all, external_df)
                external_df.to_csv(external_csv); panel_df.to_csv(panel_csv)
                return ("TPM guardada y panel actualizado.",
                        external_df.reset_index(), panel_df.reset_index())
            except Exception as e:
                return (f"Error guardando TPM: {e}", external_df.reset_index(), panel_df.reset_index())

        btn_save_tpm.click(save_tpm, inputs=[tpm_editor], outputs=[wdi_info, ext_tbl, pan_tbl])

        gr.Markdown("---")
        gr.Markdown("**CSV externo** — columnas: `Año` y alguna de `Inflación_%`, `PIB_real_%`, `USD/DOP`, `TPM_%`.")
        up_file = gr.File(label="Subir CSV", file_types=[".csv"])
        btn_load_csv = gr.Button("Cargar CSV y actualizar")

        def load_external_csv(fileobj):
            global external_df, panel_df
            if fileobj is None:
                return "Sube un CSV primero.", external_df.reset_index(), panel_df.reset_index()
            try:
                df = pd.read_csv(fileobj.name)
                if "Año" not in df.columns:
                    return "El CSV debe contener la columna 'Año'.", external_df.reset_index(), panel_df.reset_index()
                df["Año"] = df["Año"].astype(int); df = df.set_index("Año").sort_index()
                cols_ok = [c for c in ["Inflación_%","PIB_real_%","USD/DOP","TPM_%"] if c in df.columns]
                if not cols_ok:
                    return "No se encontraron columnas externas válidas en el CSV.", external_df.reset_index(), panel_df.reset_index()
                ext_new = external_df.copy(); ext_new.update(df[cols_ok]); external_df = ext_new.sort_index()
                panel_df = build_panel(years_all, external_df)
                external_df.to_csv(external_csv); panel_df.to_csv(panel_csv)
                return f"CSV cargado. Columnas actualizadas: {', '.join(cols_ok)}", external_df.reset_index(), panel_df.reset_index()
            except Exception as e:
                return f"Error leyendo CSV: {e}", external_df.reset_index(), panel_df.reset_index()

        btn_load_csv.click(load_external_csv, inputs=[up_file], outputs=[wdi_info, ext_tbl, pan_tbl])

        gr.Markdown("**Descargas**")
        gr.DownloadButton("Factores (CSV)", value=external_csv)
        gr.DownloadButton("Panel (CSV)", value=panel_csv)

    # 2) Series comparadas
    with gr.Tab("2) Series comparadas"):
        factor_dd = gr.Dropdown(factors_available, value=(factors_available[0] if factors_available else None), label="Factor externo")
        kpi_dd    = gr.Dropdown(kpi_available, value=(kpi_available[0] if kpi_available else None), label="KPI interno")
        lag_sl    = gr.Slider(0, 2, value=0, step=1, label="Rezago (años)")
        norm_cb   = gr.Checkbox(False, label="Normalizar (z-score)")
        ts_plot   = gr.Plot(label="Series (eje doble)")
        sc_plot   = gr.Plot(label="Dispersión + tendencia")
        ts_table  = gr.Dataframe(interactive=False, label="Datos usados")
        info_md   = gr.Markdown()
        def _ts(factor, kpi, lag, norm):
            return ts_compare(factor, kpi, lag=lag, normalize=norm)
        gr.Button("Comparar").click(_ts, inputs=[factor_dd, kpi_dd, lag_sl, norm_cb],
                                    outputs=[ts_plot, sc_plot, ts_table, info_md])

    # 3) Heatmap
    with gr.Tab("3) Heatmap correlaciones"):
        lag_heat = gr.Slider(0, 2, value=0, step=1, label="Rezago factores")
        heat_plot = gr.Plot(); heat_tbl  = gr.Dataframe(interactive=False, label="Matriz")
        def _heat(lag): return corr_heatmap(lag=lag)
        gr.Button("Generar").click(_heat, inputs=[lag_heat], outputs=[heat_plot, heat_tbl])

    # 4) OLS
    with gr.Tab("4) Regresión OLS"):
        dep_opts = kpi_available
        indep_opts = [c for c in FACTOR_CANDIDATES if c in panel_df.columns] + \
                     [f"{c}_lag1" for c in FACTOR_CANDIDATES if f"{c}_lag1" in panel_df.columns]
        dep_dd   = gr.Dropdown(dep_opts, value=(dep_opts[0] if dep_opts else None), label="Dependiente (KPI)")
        indep_cg = gr.CheckboxGroup(indep_opts, label="Factores (elige varios)")
        ols_tbl  = gr.Dataframe(interactive=False, label="Coeficientes")
        ols_meta = gr.Markdown(); ols_file = gr.File()
        def _ols(dep, indeps):
            res = run_ols(dep, indeps)
            if isinstance(res, str): return pd.DataFrame(), res, None
            tbl, meta, path = res; return tbl, meta, path
        gr.Button("Ejecutar OLS").click(_ols, inputs=[dep_dd, indep_cg], outputs=[ols_tbl, ols_meta, ols_file])

    # 5) PESTEL
    with gr.Tab("5) Índice de Entorno (PESTEL)"):
        years_opts = sorted(panel_df.index.tolist())
        year_dd = gr.Dropdown(years_opts, value=(years_opts[-1] if years_opts else None), label="Año de evaluación")
        with gr.Row():
            econ_w = gr.Slider(0, 100, value=25, step=1, label="Económico")
            soc_w  = gr.Slider(0, 100, value=10, step=1, label="Social")
            geo_w  = gr.Slider(0, 100, value=5,  step=1, label="Geográfico")
        with gr.Row():
            pol_w  = gr.Slider(0, 100, value=25, step=1, label="Político")
            tec_w  = gr.Slider(0, 100, value=25, step=1, label="Tecnológico")
            cul_w  = gr.Slider(0, 100, value=10, step=1, label="Cultural")
        auto_econ  = gr.Checkbox(True, label="Auto Económico (PIB, Inflación, Depreciación, TPM)")
        econ_manual= gr.Slider(-100, 100, value=0, step=1, label="Económico (manual)", interactive=True)
        econ_auto_out = gr.Number(label="Económico (auto)", interactive=False)
        soc_res = gr.Slider(-100, 100, value=0, step=1, label="Social")
        geo_res = gr.Slider(-100, 100, value=0, step=1, label="Geográfico")
        pol_res = gr.Slider(-100, 100, value=0, step=1, label="Político")
        tec_res = gr.Slider(-100, 100, value=0, step=1, label="Tecnológico")
        cul_res = gr.Slider(-100, 100, value=0, step=1, label="Cultural")
        idx_plot = gr.Plot(); idx_tbl  = gr.Dataframe(interactive=False, label="Detalle")
        idx_md   = gr.Markdown(); idx_file = gr.File()
        def _run_env(year,e_w,s_w,g_w,p_w,t_w,c_w,auto_e,e_man,s_r,g_r,p_r,t_r,c_r):
            fig, df, md, path, econ_val = env_index_calc(int(year), e_w,s_w,g_w,p_w,t_w,c_w,
                                                         auto_e, e_man, s_r,g_r,p_r,t_r,c_r)
            return fig, df, md, path, econ_val
        gr.Button("Calcular índice").click(_run_env,
            inputs=[year_dd, econ_w, soc_w, geo_w, pol_w, tec_w, cul_w,
                    auto_econ, econ_manual, soc_res, geo_res, pol_res, tec_res, cul_res],
            outputs=[idx_plot, idx_tbl, idx_md, idx_file, econ_auto_out]
        )

    # 6) Exportar PDF
    with gr.Tab("6) Exportar PDF"):
        gr.Markdown("Genera un PDF con gráficos, verticales, horizontales y ratios (sin OpenAI).")
        pdf_file = gr.File(label="PDF generado")
        def _make_pdf():
            path = generate_pdf()
            return path
        gr.Button("Generar PDF").click(_make_pdf, outputs=[pdf_file])

# Lanza una sola app
demo.queue().launch(share=True, inline=False, debug=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b3d0d75c7fb5b8e1d3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
